In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2
from glob import glob
import json
import sys
import os
import numpy as np
np.printoptions(decimals=1)
sys.path.append("../../src_clasificacion_vistas/common")

import pycimg

from pathlib import Path

import cv2

In [2]:
base='../data'
# Find all the json files recursively
json_files = glob(base + '/**/*.json', recursive=True)

print(f'Found {len(json_files)} json files')

out_dir="/home/aalbiol/Desktop/orange_views"

Path(out_dir).mkdir(parents=True, exist_ok=True)

Found 1669 json files


In [3]:
def getViewId(id):
    partes=id.split("_")
    return int(partes[1])

In [4]:
defectos=["cracks",
  "damage_by_pliers",
  "deformed_peduncle",
  "green",
  "hollow",
  "light_scars",
   "oil_spots",
  "plu_stickers",
  "reddish",
  "rotten",
  "scars",
  "skin_breakdown",
  "surface",
]

defectos=["insect",
        "mechanical_damage"]

In [7]:
np.set_printoptions(precision=1)
json_files = glob(base + '/**/*.json', recursive=True)



listas_positivos={} # Casos po
listas_negativos={}
for defecto in defectos:
    listas_positivos[defecto]=[]
    listas_negativos[defecto]=[]
    
for json_file in json_files:

    with open(json_file, 'r') as f:
        data = json.load(f)
        if "views_annotations" not in data: # Ignoro los pocos que no tienen vistas
            continue
            
        anotaciones_vistas=data["views_annotations"]
        #print(anotaciones_vistas)
        anots_array=[]
        for nombre_vista,anotaciones_vista in anotaciones_vistas.items():
            view_number=getViewId(nombre_vista)
            for defecto,valor in anotaciones_vista.items():
                if defecto not in defectos:
                    continue
                label=int(valor)
                if label==1:
                    listas_positivos[defecto].append((json_file,view_number))
                else:
                    listas_negativos[defecto].append((json_file,view_number))



In [8]:
for defecto,lista in listas_positivos.items():
    print(f'{defecto}: {len(lista)} positivos')

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")    
for defecto,lista in listas_negativos.items():
    print(f'{defecto}: {len(lista)} negativos')
    

insect: 126 positivos
mechanical_damage: 226 positivos
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
insect: 15857 negativos
mechanical_damage: 15757 negativos


In [ ]:
for defecto,lista in listas_positivos.items():
    print(f'{defecto}: {len(lista)} positivos')



insect: 126 positivos
mechanical_damage: 226 positivos


In [10]:
from tqdm import tqdm
for defecto in defectos:
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(defecto,len(listas_positivos[defecto]))
    lista_positivos=listas_positivos[defecto]
    #lista_negativos=listas_negativos_seleccionados[defecto]
    
    # Crear un directorio para cada defecto
    
    positive_dir=out_dir+"/positive/"+defecto
    #negative_dir=out_dir+"/negative/"+defecto
    Path(positive_dir).mkdir(parents=True, exist_ok=True)
    #Path(negative_dir).mkdir(parents=True, exist_ok=True)
    print(" >>> Positive")
    for vista in tqdm(lista_positivos):
        json_file,view_number=vista
        npzfilename=json_file.replace(".json",".npz")
        imagenes=pycimg.npzread_torch(npzfilename,json_file,channel_list=['R','G','B'])
        imagen=imagenes[view_number].numpy()
        imagen=imagen.transpose((1,2,0))
        imagen=(255*imagen).astype(np.uint8)
        imagen_bgr=imagen[:,:,::-1]
        imagen_bgr=cv2.resize(imagen_bgr,(224,224))
        #print(f'Vista positiva: {json_file} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        
        basename=os.path.basename(json_file)
        basename=basename.replace(".json","")
        basename=f'{basename}_{view_number:02d}.jpg'
        #print(f'Vista positiva: {basename} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        outname=os.path.join(positive_dir,basename)
        cv2.imwrite(outname,imagen_bgr)
 
        


    

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
insect 126
 >>> Positive


  0%|          | 0/126 [00:00<?, ?it/s]

100%|██████████| 126/126 [00:01<00:00, 76.18it/s]


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
mechanical_damage 226
 >>> Positive


100%|██████████| 226/226 [00:05<00:00, 40.05it/s]
